# 基本操作

In [6]:
import xlwings as xw
import matplotlib.pyplot as plt
import datetime as dt

# 关联Book
wb = xw.Book('./收入-DAU预估模型/收入预估.xlsx')

# 选择某个Sheet，选中某个Sheet
sht = wb.sheets['Test']
sht.select()

# 遍历Book中的所有Sheet
for eachSht in wb.sheets:
    print(eachSht.name)
# print(type(wb.sheets))
# print(type(sht))
# print(wb.sheets[0].name)

# 操作单元格
# 通过A1B1格式调用
sht.range('A1').value = 'A1'
# 通过行数+列数格式调用
sht.range((1,2)).value = 'B1'
for i in range(1, 12):
    sht.range((i,2)).value = dt.datetime(2019, i, 1)
sht.range((1,1)).options(transpose = True).value = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# 保存当前修改
wb.save()

# 关闭一个Book
# wb.close()

模拟计算
月活跃
留存衰减曲线
收入误差
历史留存数据
历史收入数据
Test


# AI属性规划

In [2]:
import json
import xlrd
import xlwt

def uniquelist(original_list):
    target_list = []
    for i in original_list:
        if i not in target_list:
            target_list.append(i)
    return target_list

def removeall(slist, r):
    while r in slist:
        slist.remove(r)
    return slist

def dict_reverse(dict0):
    dict1 = {}
    keylist = list(dict0.keys())
    valuelist = list(dict0.values())
    if len(keylist) != 0:
        keylist.reverse()
        valuelist.reverse()
        for i in range(len(keylist)):
            dict1[keylist[i]] = valuelist[i]
        return dict1
    else:
        return dict0

final_file=open('json.txt',mode='w')

TankList = []
Tank = {}
TankIdList = []
battleTankComponentInfo = {"isNeedReceive":True}
TankPartLists = []
TankActiveSkillLists = []
TankPassiveSkillLists = []

Workbook = xlrd.open_workbook('AI属性规划.xlsx')
IdSheet = Workbook.sheet_by_name(u'ID')
SkillSheet = Workbook.sheet_by_name(u'Skill')
JSONSheet = Workbook.sheet_by_name(u'JSON')
# print(IdSheet.cell(2,0).value)
StartRow = 5
TypeRow = 1
KeyRow = 4
TankIdRow = 2
PartNumRow = 3

TypeList = IdSheet.row_values(TypeRow)
SkillTypeList = SkillSheet.row_values(TypeRow)
RowNum = len(IdSheet.col_values(1)) - StartRow
# print("RowNum", RowNum)

# print("TypeList:")
# print(TypeList)

for i in range(len(TypeList)):
    if TypeList[i] == "tank":
        TankIdList.append(IdSheet.cell(TankIdRow,i).value)
        TankIdList = uniquelist(TankIdList)

# print(TankIdList)

for i in range(len(TankIdList)):
    TankList.append({})
    TankPartLists.append([{},{},{},{},{},{}])
    TankActiveSkillLists.append([{},{}])
    TankPassiveSkillLists.append([{},{},{},{},{},{}])

print(TankPartLists)

for row in range(RowNum):
    # print("row",row)
    for TankNum in range(len(TankIdList)):
        for i in range(len(TypeList)):
            # print(i)
            if (IdSheet.cell(TankIdRow,i).value == TankIdList[TankNum]) and (TypeList[i] == "part"):
                if IdSheet.cell(StartRow + row,i).value != "":
                    # print(TankNum)
                    TankPartLists[TankNum][int(IdSheet.cell(PartNumRow,i).value) - 1]["not_initial"] = 1
                    TankPartLists[TankNum][int(IdSheet.cell(PartNumRow,i).value) - 1]["equip_num"] = 1
                    TankPartLists[TankNum][int(IdSheet.cell(PartNumRow,i).value) - 1]["total_num"] = 1                    
                    TankPartLists[TankNum][int(IdSheet.cell(PartNumRow,i).value) - 1]["cur_num"] = 1

        for i in range(len(TypeList)):
            if (IdSheet.cell(TankIdRow,i).value == TankIdList[TankNum]) and (TypeList[i] == "part"):
                if IdSheet.cell(StartRow + row,i).value != "":                    
                    TankPartLists[TankNum][int(IdSheet.cell(PartNumRow,i).value) - 1][IdSheet.cell(KeyRow,i).value] = int(IdSheet.cell(StartRow + row,i).value)

        for i in range(len(TypeList)):
            if (IdSheet.cell(TankIdRow,i).value == TankIdList[TankNum]) and (TypeList[i] == "part"):
                if IdSheet.cell(StartRow + row,i).value != "":                    
                    TankPartLists[TankNum][int(IdSheet.cell(PartNumRow,i).value) - 1]["tank_part_id"] = int(IdSheet.cell(PartNumRow,i).value +  (IdSheet.cell(TankIdRow,i).value - 1) * 6)
        
        for i in range(len(TankPartLists[TankNum])):
            TankPartLists[TankNum][i] = dict_reverse(TankPartLists[TankNum][i])
        
        for i in range(len(SkillTypeList)):
            if (SkillSheet.cell(TankIdRow,i).value == TankIdList[TankNum]) and (SkillTypeList[i] == "askill"):
                if SkillSheet.cell(StartRow + row,i).value != "":
                    TankActiveSkillLists[TankNum][int(SkillSheet.cell(PartNumRow,i).value) - 1][SkillSheet.cell(KeyRow,i).value] = int(SkillSheet.cell(StartRow + row,i).value)
            if (SkillSheet.cell(TankIdRow,i).value == TankIdList[TankNum]) and (SkillTypeList[i] == "pskill"):
                if SkillSheet.cell(StartRow + row,i).value != "":
                    TankPassiveSkillLists[TankNum][int(SkillSheet.cell(PartNumRow,i).value) - 1][SkillSheet.cell(KeyRow,i).value] = int(SkillSheet.cell(StartRow + row,i).value)

    
    for TankNum in range(len(TankIdList)):
        removeall(TankPartLists[TankNum],{})
        removeall(TankActiveSkillLists[TankNum],{})
        removeall(TankPassiveSkillLists[TankNum],{})

    for TankNum in range(len(TankIdList)):
        TankList[TankNum]["tankID"] = int(TankIdList[TankNum])
        for i in range(len(TypeList)):
            if (IdSheet.cell(TankIdRow,i).value == TankIdList[TankNum]) and (TypeList[i] == "tank"):            
                TankList[TankNum][IdSheet.cell(KeyRow,i).value] = int(IdSheet.cell(StartRow + row,i).value)
        TankList[TankNum]["battleTankSerial"] = int(TankIdList[TankNum])
        TankList[TankNum]["tankPower"] = 0
        TankList[TankNum]["battleTankComponentInfo"] = battleTankComponentInfo
        TankList[TankNum]["tankPartLists"] = TankPartLists[TankNum]
        if TankActiveSkillLists[TankNum] != []:
            TankList[TankNum]["tankActiveSkillList"] = TankActiveSkillLists[TankNum]
        if TankPassiveSkillLists[TankNum] != []:
            TankList[TankNum]["tankPassiveSkillList"] = TankPassiveSkillLists[TankNum]
        # print(Tank)
    

    final = {"battleTanks":TankList}
    json_line = json.dumps(final)
    json_line = json_line + "\n"
    # print(json_line)
    final_file.writelines(json_line)

    for i in range(len(TankIdList)):
        TankList[i] = {}
        TankPartLists[i] = [{},{},{},{},{},{}]
        TankActiveSkillLists[i] = [{},{}]
        TankPassiveSkillLists[i] = [{},{},{},{},{},{}]


[[{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}]]


# 宝箱规划

In [3]:
import xlwings as xw
import pandas as pd
import numpy as np

wb = xw.Book('./宝箱填表.xlsx')

srcSht = wb.sheets['数据源']
dstSht = wb.sheets['数据导出']
srcData = pd.read_excel('宝箱填表.xlsx', '数据源', index_col=None, na_values=['NA'])
# srcData
dropItem = np.array(srcData['掉落物'])
chestName = np.array(srcData['宝箱名称'])
dropWeight = np.array(srcData['掉落权重'])
dropNum = np.array(srcData['掉落数量'])
dropTimes = np.array(srcData['抽取次数'])

def toArray(arrayName):
    for i in range(len(arrayName)):
        arrayName[i] = arrayName[i].replace('[','')
        arrayName[i] = arrayName[i].replace(']','')
        arrayName[i] = arrayName[i].replace('(','')
        arrayName[i] = arrayName[i].replace(')','')
        arrayName[i] = np.array(arrayName[i].split(','))
        len(arrayName[i])
        arrayName[i]

toArray(dropItem)
toArray(dropWeight)
toArray(dropNum)
sequalNo = 1

dropNum1 = np.array(dropNum)
dropNum2 = np.array(dropNum)

def toOdd(arrayName):
    for i in range(len(arrayName)):
        if i % 2 == 0:
            dropNum1.add(arrayName[i])

def toEven(arrayName):
    for i in range(len(arrayName)):
        if i % 2 == 1:
            dropNum1.add(arrayName[i])

for i in range(len(dropNum)):
    dropNum1[i] = dropNum1[i][::2]
    dropNum2[i] = dropNum2[i][1::2]

# dropNum1
# dropNum2


for i in range(len(dropItem)):
    dstSht.range((sequalNo,2)).options(transpose = True).value = dropItem[i]
    dstSht.range((sequalNo,3)).options(transpose = True).value = dropWeight[i]
    chestName[i] = np.array(chestName[i])
    chestName[i] = chestName[i].repeat(len(dropItem[i]))
    dstSht.range((sequalNo,1)).options(transpose = True).value = chestName[i]
    dstSht.range((sequalNo,4)).options(transpose = True).value = dropNum1[i]
    dstSht.range((sequalNo,5)).options(transpose = True).value = dropNum2[i]
    sequalNo = sequalNo + len(dropItem[i])
print("done")

done


# 导出宝箱掉落配置

In [1]:
import xlwings as xw
import pandas as pd
import numpy as np

wb = xw.Book('./宝箱填表.xlsx')
chestList = ['钥匙宝箱','出售宝箱1','出售宝箱2','出售宝箱3','碎片宝箱','活跃小箱','活跃中箱','活跃大箱','活跃巨箱']
chestListS = ['稀有宝箱','稀有宝箱保底抽']
rankList = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
chestNameList = []
itemList = []
weightList = []
numList = []
sheetNameList = []
exportSht = wb.sheets['导出汇总']
randomSht = wb.sheets[chestList[0]]

for chest in chestListS:
    rawData = pd.read_excel('宝箱填表.xlsx', chest, index_col=None, na_values=['NA'])
    chestNameCol = np.array(rawData['掉落池名称'])
    rankCol = np.array(rawData['联赛等级'])
    itemCol = np.array(rawData['物品id'])
    weightCol = np.array(rawData['权重'])
    numCol = np.array(rawData['数量导出'])
    allArray = np.column_stack((chestNameCol, rankCol, itemCol, weightCol, numCol))
    # allArray[:, 0]
    for i in range(len(rankList)):
        rankMask = np.in1d(allArray[:, 1], rankList[i])
        rankArray = allArray[rankMask]

        chestNameRankArray = str(rankArray[:,0][0]).replace(' ','')

        itemRankArray = str(list(rankArray[:,2])).replace(' ','')

        weightRankArray = str(list(rankArray[:,3])).replace(' ','')

        numRankArray0 = str(list(rankArray[:,4])).replace(' ','')
        numRankArray = numRankArray0.replace('\'','')
        
        sheetNameArray = chest

#         print("allArray",allArray)
#         print("rankArray",rankArray)

    #     z = str((y[:,1]))
    #     k = z.replace(' ',',')
        chestNameList.append(chestNameRankArray)
        itemList.append(itemRankArray)
        weightList.append(weightRankArray)
        numList.append(numRankArray)
        sheetNameList.append(sheetNameArray)
        
    print(chest)

exportSht.range((1,1)).options(transpose = True).value = chestNameList
exportSht.range((1,2)).options(transpose = True).value = itemList
exportSht.range((1,3)).options(transpose = True).value = weightList
exportSht.range((1,4)).options(transpose = True).value = numList
exportSht.range((1,5)).options(transpose = True).value = sheetNameList


# rankArray[:,0][0]

# print(exportList)

# print(list(item_export))
# print(type(list(item_export)))
# x = str(list(item_export))
# temp = x.replace(' ','')
# exportSht.range(1,1).value = temp
# print(temp)
# print(type(temp))

稀有宝箱
稀有宝箱保底抽


In [8]:
import xlwings as xw
import pandas as pd
import numpy as np

wb = xw.Book('./宝箱填表.xlsx')

exportSht = wb.sheets['导出汇总']


exportSht.range((1,1)).options(transpose = True).value = 1